engine 객체에 db api를 통해 뭐라는거야..\
객체와 sql문을 연결해주는 애ㅏㄱ 필요 -> 스키마와 타입이 이런 역할을 함\
sql exp\
타입에는 제약조건들, 타입이 들어가 있다.\
engine으로부터 커넥션을 받는다.\
엔진에는 starting point가 있다.\
SQLite는 파일DB라 port없다.\
lazy connection: 저 당시에 접근이 이루어지는게 아니라 나중에 일어난다.\


In [1]:
import sqlalchemy

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey
from sqlalchemy.types import Integer, String

In [5]:
# starting point
engine = create_engine('sqlite:///orml.db', echo=True)

In [7]:
meta = MetaData() # bing 옵션 = engine 연결

In [8]:
meta.tables

FacadeDict({})

In [9]:
Table('T_USER', meta,
      Column('PK', Integer, primary_key=True),
      Column('NAME', String, nullable=False))

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)

In [10]:
# 테이블 객체가 하나 생성 -> meta 등록, 아직은 물리적으로 DB에 반영 안 됨.XXX

In [11]:
meta.tables

FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)})

In [12]:
meta.bind = engine

In [13]:
meta.create_all()

2023-03-07 09:42:15,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 09:42:15,129 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-07 09:42:15,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 09:42:15,132 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2023-03-07 09:42:15,133 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 09:42:15,137 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" VARCHAR NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-03-07 09:42:15,137 INFO sqlalchemy.engine.Engine [no key 0.00091s] ()
2023-03-07 09:42:15,147 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
from sqlalchemy.sql import select, insert

In [17]:
print(select(meta.tables['T_USER']))

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"


In [18]:
# (?, ?) 큐마크 스타일
print(meta.tables['T_USER'].insert())

INSERT INTO "T_USER" ("PK", "NAME") VALUES (?, ?)


In [19]:
User = meta.tables['T_USER']

In [20]:
print(insert(User), select(User), User.update())

INSERT INTO "T_USER" ("PK", "NAME") VALUES (?, ?) SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" UPDATE "T_USER" SET "PK"=?, "NAME"=?


In [21]:
print(meta.tables['T_USER'].insert().values(NAME='아무나'))

INSERT INTO "T_USER" ("NAME") VALUES (?)


In [22]:
print(User.insert().values(NAME='아무나').compile())

INSERT INTO "T_USER" ("NAME") VALUES (?)


In [23]:
print(User.insert().values(NAME='아무나').compile().params)

{'NAME': '아무나'}


In [24]:
con = engine.connect()

In [25]:
insert(User, bind=engine).values(NAME='아무나')

In [26]:
cur = con.execute(insert(User).values(NAME='아무나'))

2023-03-07 09:55:07,530 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2023-03-07 09:55:07,533 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ('아무나',)
2023-03-07 09:55:07,538 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
cur = con.execute(User.select())

2023-03-07 09:57:01,074 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"
2023-03-07 09:57:01,082 INFO sqlalchemy.engine.Engine [generated in 0.00225s] ()


In [28]:
cur.fetchall()

[(1, '아무나')]

In [30]:
from sqlalchemy.sql import and_, or_

In [32]:
print(User.c.PK == 1)

"T_USER"."PK" = :PK_1


In [31]:
print(and_(User.c.PK == 1, User.c.NAME == 1))

"T_USER"."PK" = :PK_1 AND "T_USER"."NAME" = :NAME_1


In [33]:
Table('T_ADDRESS', meta,
     Column('PK', Integer, primary_key=True),
     Column('ADDR', String),
     Column('FK', Integer, nullable=False))

Table('T_ADDRESS', MetaData(bind=Engine(sqlite:///orml.db)), Column('PK', Integer(), table=<T_ADDRESS>, primary_key=True, nullable=False), Column('ADDR', String(), table=<T_ADDRESS>), Column('FK', Integer(), table=<T_ADDRESS>, nullable=False), schema=None)

In [35]:
len(meta.tables), meta.tables

(2,
 FacadeDict({'T_USER': Table('T_USER', MetaData(bind=Engine(sqlite:///orml.db)), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None), 'T_ADDRESS': Table('T_ADDRESS', MetaData(bind=Engine(sqlite:///orml.db)), Column('PK', Integer(), table=<T_ADDRESS>, primary_key=True, nullable=False), Column('ADDR', String(), table=<T_ADDRESS>), Column('FK', Integer(), table=<T_ADDRESS>, nullable=False), schema=None)}))

In [36]:
meta.bind

Engine(sqlite:///orml.db)

In [37]:
meta.create_all()

2023-03-07 10:02:49,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 10:02:49,245 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-07 10:02:49,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 10:02:49,249 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2023-03-07 10:02:49,251 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 10:02:49,251 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ADDRESS")
2023-03-07 10:02:49,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 10:02:49,257 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ADDRESS" (
	"PK" INTEGER NOT NULL, 
	"ADDR" VARCHAR, 
	"FK" INTEGER NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-03-07 10:02:49,257 INFO sqlalchemy.engine.Engine [no key 0.00114s] ()
2023-03-07 10:02:49,268 INFO sqlalchemy.engine.Engine COMMIT


In [38]:
Addr = meta.tables['T_ADDRESS']

In [39]:
type(User), type(Addr)

(sqlalchemy.sql.schema.Table, sqlalchemy.sql.schema.Table)

In [40]:
print(select([User, Addr]))

SELECT "T_USER"."PK", "T_USER"."NAME", "T_ADDRESS"."PK" AS "PK_1", "T_ADDRESS"."ADDR", "T_ADDRESS"."FK" 
FROM "T_USER", "T_ADDRESS"


In [41]:
from sqlalchemy.sql import join

In [42]:
join(User, Addr, User.c.PK==Addr.c.FK)

<sqlalchemy.sql.selectable.Join at 0x29a96283d90; Join object on T_USER(2862973957888) and T_ADDRESS(2862967672688)>

In [46]:
print(select([User.c.NAME, Addr.c.ADDR])\
     .select_from(join(User, Addr, User.c.PK==Addr.c.FK)))

SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"


In [47]:
sql = select([User.c.NAME, Addr.c.ADDR])\
     .select_from(join(User, Addr, User.c.PK==Addr.c.FK))
cur = con.execute(sql)
cur.fetchall()

2023-03-07 10:49:08,115 INFO sqlalchemy.engine.Engine SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"
2023-03-07 10:49:08,116 INFO sqlalchemy.engine.Engine [generated in 0.00205s] ()


[]

In [49]:
print(select([User.c.NAME, Addr.c.ADDR])\
     .select_from(join(User, Addr, User.c.PK==Addr.c.FK))\
     .where(User.c.PK == 1))

SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK" 
WHERE "T_USER"."PK" = ?


In [54]:
# print(User.select().where(User.c.PK==1))
# print(select(User))
FK = con.execute(User.select().where(User.c.PK==1)).fetchone()[0]
                # 1. User 객체 - SELECT - WHERE
                # 2. engine의 connection pool에서 관리하는 connection -> con.execute
                # 3. 2.의 결과가 resultproxy로 온다. -> cursor
                # 4. 3.의 결과(DB의 실제 데이터)를 getchone(1행)[0 => 1열]
            
# print(insert(Addr).values(ADDR='아무거나', FK=1))
cur = con.execute(insert(Addr).values(ADDR='아무거나', FK=FK))
cur.lastrowid

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"


In [55]:
con.close()
meta.clear()
engine.dispose()

In [56]:
engine = create_engine('sqlite:///orm1.db', echo=True)
meta = MetaData(engine)
con = engine.connect()

In [57]:
meta.tables

FacadeDict({})

In [58]:
Table('T_USER', meta,
     Column('PK', Integer))

Table('T_USER', MetaData(bind=Engine(sqlite:///orm1.db)), Column('PK', Integer(), table=<T_USER>), schema=None)

In [ ]:
con.execute

In [59]:
User

Table('T_USER', MetaData(bind=Engine(sqlite:///orml.db)), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)

In [63]:
meta.clear()

In [64]:
Table('T_USER', meta,
     Column('Pk', Integer),
     Column('TEST', String),
     extend_existing=True)

Table('T_USER', MetaData(bind=Engine(sqlite:///orm1.db)), Column('Pk', Integer(), table=<T_USER>), Column('TEST', String(), table=<T_USER>), schema=None)

In [65]:
con.close()
meta.clear()
engine.dispose()

In [66]:
# 프랜차이즈
engine = create_engine('sqlite:///test1.db', echo=True)
meta = MetaData(engine)
con = engine.connect()

In [67]:
# city, part, sells, supplier
# pk, name  fk1, fk2, price,  fk
Table('city', meta,
     Column('pk', Integer, primary_key=True),
     Column('name', String), extend_existing=True)

Table('part', meta,
     Column('pk', Integer, primary_key=True),
     Column('name', String), extend_existing=True)

Table('supplier', meta,
     Column('pk', Integer, primary_key=True),
     Column('name', String),
     Column('fk', Integer), extend_existing=True)

Table('sells', meta,
     Column('fk1', Integer),
     Column('fk2', Integer),
     Column('price', Integer), extend_existing=True)

Table('sells', MetaData(bind=Engine(sqlite:///test1.db)), Column('fk1', Integer(), table=<sells>), Column('fk2', Integer(), table=<sells>), Column('price', Integer(), table=<sells>), schema=None)

In [68]:
meta.create_all()

2023-03-07 11:27:14,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 11:27:14,629 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("city")
2023-03-07 11:27:14,631 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 11:27:14,632 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("part")
2023-03-07 11:27:14,633 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 11:27:14,635 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("supplier")
2023-03-07 11:27:14,635 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 11:27:14,637 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sells")
2023-03-07 11:27:14,638 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 11:27:14,639 INFO sqlalchemy.engine.Engine COMMIT


In [69]:
con.execute(select(meta.tables['sells'])).fetchall()

2023-03-07 11:27:24,095 INFO sqlalchemy.engine.Engine SELECT sells.fk1, sells.fk2, sells.price 
FROM sells
2023-03-07 11:27:24,096 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()


[]

In [70]:
city = meta.tables['city']
supplier = meta.tables['supplier']
part = meta.tables['part']
sells = meta.tables['sells']

In [71]:
sql = select([city.c.name, supplier.c.name])\
            .select_from(join(city, supplier, city.c.pk==supplier.c.fk))
con.execute(sql).fetchall()

2023-03-07 11:27:28,362 INFO sqlalchemy.engine.Engine SELECT city.name, supplier.name AS name_1 
FROM city JOIN supplier ON city.pk = supplier.fk
2023-03-07 11:27:28,363 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()


[('성북구', '안암1호점'), ('성북구', '안암2호점'), ('성북구', '종암1호점')]

* supplier - sells - part 3개 조인
* city - supplier - sells - part 4개 조인
* select할 columns 지정도 해보고
* where 조건도 표현해보고,
* groupby, orderby => 할 수 있음 해보고 (PPT에 있음)

1. engine = create_engine(주소) 엔진만들고
2. meta = MetaData 데이터만들고
3. con = engine ??
* engine한테 일 하라고 시키는 걸 binding이라고 한다.
* 이 순서대로 해야 에러가 안 난다.

In [79]:
con.close()
meta.clear()
engine.dispose()

In [80]:
# playlist
engine = create_engine('sqlite:///playlist.db', echo=True)
meta = MetaData(engine)
con = engine.connect()

In [81]:
meta.reflect(engine)

2023-03-07 12:07:46,738 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-03-07 12:07:46,739 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 12:07:46,742 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("album")
2023-03-07 12:07:46,743 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 12:07:46,745 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-07 12:07:46,746 INFO sqlalchemy.engine.Engine [raw sql] ('album',)
2023-03-07 12:07:46,748 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("album")
2023-03-07 12:07:46,749 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 12:07:46,751 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("album")
2023-03-07 12:07:46,751 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 12:07:46,752 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sq

In [82]:
meta.tables.keys()

dict_keys(['album', 'artist', 'genre', 'track'])

In [83]:
album = meta.tables['album']
artist = meta.tables['artist']

In [85]:
sql = select([album.c.name, artist.c.name])\
    .select_from(join(album, artist, album.c.fk==artist.c.pk))

In [86]:
con.execute(sql).fetchall()

2023-03-07 12:08:22,507 INFO sqlalchemy.engine.Engine SELECT album.name, artist.name AS name_1 
FROM album JOIN artist ON album.fk = artist.pk
2023-03-07 12:08:22,507 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()


[]

In [87]:
album.columns.keys(), artist.columns.keys()

(['pk', 'name', 'fk'], ['pk', 'name'])

In [88]:
con.execute(album.select()).fetchall()

2023-03-07 12:09:07,158 INFO sqlalchemy.engine.Engine SELECT album.pk, album.name, album.fk 
FROM album
2023-03-07 12:09:07,171 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ()


[]

In [89]:
con.execute(insert(album), [{'name':'아무개1', 'fk':1},
                           {'name':'아무개2', 'fk':2}])

2023-03-07 12:11:19,385 INFO sqlalchemy.engine.Engine INSERT INTO album (name, fk) VALUES (?, ?)
2023-03-07 12:11:19,385 INFO sqlalchemy.engine.Engine [generated in 0.00140s] (('아무개1', 1), ('아무개2', 2))
2023-03-07 12:11:19,401 INFO sqlalchemy.engine.Engine COMMIT


In [90]:
con.execute(insert(artist), [{'name':'가수1'},
                           {'name':'가수2'}])

2023-03-07 12:11:49,550 INFO sqlalchemy.engine.Engine INSERT INTO artist (name) VALUES (?)
2023-03-07 12:11:49,552 INFO sqlalchemy.engine.Engine [generated in 0.00185s] (('가수1',), ('가수2',))
2023-03-07 12:11:49,560 INFO sqlalchemy.engine.Engine COMMIT


In [92]:
sql = select([album.c.name, artist.c.name])\
            .select_from(join(album, artist, album.c.fk==artist.c.pk))
con.execute(sql).fetchall()

2023-03-07 12:15:30,467 INFO sqlalchemy.engine.Engine SELECT album.name, artist.name AS name_1 
FROM album JOIN artist ON album.fk = artist.pk
2023-03-07 12:15:30,469 INFO sqlalchemy.engine.Engine [cached since 427.9s ago] ()


[('아무개1', '가수1'), ('아무개2', '가수2')]

In [93]:
print(insert(artist))

INSERT INTO artist (pk, name) VALUES (?, ?)


In [94]:
print(insert(artist, {'name':'한소희'})),\
print(insert(artist, {'name':'한소희'}).compile().params)

INSERT INTO artist (name) VALUES (?)
{'name': '한소희'}


(None, None)

In [95]:
con.execute(artist.update().values(name='가수1').where(artist.c.pk==1))

2023-03-07 12:17:26,664 INFO sqlalchemy.engine.Engine UPDATE artist SET name=? WHERE artist.pk = ?
2023-03-07 12:17:26,666 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ('가수1', 1)
2023-03-07 12:17:26,666 INFO sqlalchemy.engine.Engine COMMIT


In [96]:
print(artist.update().values(name='가수10').where(artist.c.pk==1))

UPDATE artist SET name=? WHERE artist.pk = ?


In [97]:
con.execute(artist.select()).fetchall()

2023-03-07 12:18:05,924 INFO sqlalchemy.engine.Engine SELECT artist.pk, artist.name 
FROM artist
2023-03-07 12:18:05,926 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ()


[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4'), (5, '가수1'), (6, '가수2')]

multiple조인할 때는 on절을 안 쓴다 그 이유는 뭐 알아서 알기 때문에..?

어느 시점에 DB를 호출했느냐가 중요하다\
oop관점에서 코딩이 가능해진다\
인스턴스만 생성하면 된다.\
=ORM\
세션이 값을 감시하다